In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load AAPL stock price data
df = pd.read_csv('AAPL.csv')  # Replace 'AAPL.csv' with the actual file name

# Consider only 'Close' prices
df = df[['Close']]

# Visualize stock price history
plt.figure(figsize=(16, 8))
plt.title('AAPL Stock Price History')
plt.plot(df['Close'])
plt.xlabel('Date')
plt.ylabel('Close Price (USD)')
plt.show()

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(np.array(df).reshape(-1, 1))

# Create training dataset
train_data = df_scaled[0:int(np.ceil(len(df_scaled) * .95)), :]

# Create x_train and y_train datasets
x_train, y_train = [], []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape data for LSTM model
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(x_train, y_train, batch_size=1, epochs=1)

# Create testing dataset
test_data = df_scaled[int(np.ceil(len(df_scaled) * .95)) - 60:, :]

# Create x_test and y_test datasets
x_test, y_test = [], df['Close'][int(np.ceil(len(df_scaled) * .95)):].values
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

x_test = np.array(x_test)

# Reshape data for LSTM model
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Get predicted stock prices
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Visualize predictions
train = df[:int(np.ceil(len(df_scaled) * .95))]
valid = df[int(np.ceil(len(df_scaled) * .95)):]
valid['Predictions'] = predictions

plt.figure(figsize=(16, 8))
plt.title('AAPL Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Close Price (USD)')
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Validation', 'Predictions'])
plt.show()